In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

In [ ]:
from abc import ABC, abstractmethod
from typing import List, Tuple, Any, Dict
import numpy as np

from rank_bm25 import BM25Okapi
from kiwipiepy import Kiwi


class Indexer(ABC):
    @abstractmethod
    def search(self, query_text: Any, top_k: int):
        pass


class KoreanBM25Indexer(Indexer):
    def __init__(self, corpus: List[Dict[str, Any]], top_k: int = 5):
        """
        corpus format:
        [
          {
            "id": chunk_id,
            "chunked_text": "...",
            "original_docid": ...
          },
          ...
        ]
        """
        self.top_k = top_k

        # 🔹 chunk id와 텍스트 분리
        self.doc_ids = [item["id"] for item in corpus]
        self.texts = [item["chunked_text"] for item in corpus]

        # 한국어 형태소 토크나이저
        self.kiwi = Kiwi()

        # 문서 토크나이즈
        tokenized_corpus = [self._tokenize(text) for text in self.texts]

        # BM25 인덱스 생성
        self.bm25 = BM25Okapi(tokenized_corpus)

    def _tokenize(self, text: str) -> List[str]:
        # 형태소 단위 토큰
        return [tok.form for tok in self.kiwi.tokenize(text)]

    def search(self, query_text: Any, top_k: int = None):
        if top_k is None:
            top_k = self.top_k

        if isinstance(query_text, str):
            q_tokens = self._tokenize(query_text)
            scores = np.asarray(self.bm25.get_scores(q_tokens))

            top_idx = np.argsort(scores)[::-1][:top_k]
            doc_ids = [self.doc_ids[i] for i in top_idx]
            top_scores = scores[top_idx]

            return doc_ids, top_scores

        elif isinstance(query_text, list):
            return [self.search(q, top_k) for q in query_text]

        else:
            raise NotImplementedError("Input type not str nor list.")


In [5]:
from pipeline.dataset.news import get_news_dataset

dataset = get_news_dataset()

In [9]:
from pipeline.retriever.embed import chunk_korean_sentence

In [12]:
points = []

for doc_idx, _ in enumerate(dataset['text']):
    
    chunk_texts = chunk_korean_sentence(dataset[doc_idx]['text'], chunk_size=1000, chunk_overlap=200)
    
    for chunk_idx, chunk_text in enumerate(chunk_texts):
        point = {
            "id": doc_idx*1000+chunk_idx,
            'chunked_text' : chunk_text,
            'original_docid' : doc_idx
        }
        points.append(point)

In [26]:
points[0:4]

[{'id': 0,
  'chunked_text': '추경호 중기 수출지원 총력 무역금융 40조 확대\n앵커 정부가 올해 하반기 우리 경제의 버팀목인 수출 확대를 위해 총력을 기울이기로 했습니다. 특히 수출 중소기업의 물류난 해소를 위해 무역금융 규모를 40조 원 이상 확대하고 물류비 지원과 임시선박 투입 등을 추진하기로 했습니다. 류환홍 기자가 보도합니다. 기자 수출은 최고의 실적을 보였지만 수입액이 급증하면서 올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록했습니다. 정부가 수출확대에 총력을 기울이기로 한 것은 원자재 가격 상승 등 대외 리스크가 가중되는 상황에서 수출 증가세 지속이야말로 한국경제의 회복을 위한 열쇠라고 본 것입니다. 추경호 경제부총리 겸 기획재정부 장관 정부는 우리 경제의 성장엔진인 수출이 높은 증가세를 지속할 수 있도록 총력을 다하겠습니다. 우선 물류 부담 증가 원자재 가격 상승 등 가중되고 있는 대외 리스크에 대해 적극 대응하겠습니다. 특히 중소기업과 중견기업 수출 지원을 위해 무역금융 규모를 연초 목표보다 40조 원 늘린 301조 원까지 확대하고 물류비 부담을 줄이기 위한 대책도 마련했습니다. 이창양 산업통상자원부 장관 국제 해상운임이 안정될 때까지 월 4척 이상의 임시선박을 지속 투입하는 한편 중소기업 전용 선복 적재 용량 도 현재보다 주당 50TEU 늘려 공급하겠습니다. 하반기에 우리 기업들의 수출 기회를 늘리기 위해 2 500여 개 수출기업을 대상으로 해외 전시회 참가를 지원하는 등 마케팅 지원도 벌이기로 했습니다. 정부는 또 이달 중으로 반도체를 비롯한 첨단 산업 육성 전략을 마련해 수출 증가세를 뒷받침하고 에너지 소비를 줄이기 위한 효율화 방안을 마련해 무역수지 개선에 나서기로 했습니다. YTN 류환홍입니다.',
  'original_docid': 0},
 {'id': 1,
  'chunked_text': '올해 상반기 우리나라 무역수지는 역대 최악인 103억 달러 적자를 기록한 가운데, 정부가 하반기에